In [55]:
# Provide the log file and run all the cells in the notebook
# LOG_FILE = "O2PL-log.txt"
LOG_FILE = "SS2PL-log.txt"

In [56]:
# Read the log file
with open(LOG_FILE, "r") as f:
    lines = f.readlines()
    lines = [line.rstrip('\n') for line in lines]

print(len(lines))
lines[-1]

66066


'Transaction 1994 commits at time 1745172198082192'

In [57]:
operations = []

for line in lines:
    words = line.split()
    trans_id = int(words[1])
    if words[2] == 'reads':
        op = 'r'
        tm = int(words[7])
    elif words[2] == 'writes':
        op = 'w'
        tm = int(words[7])
    elif words[2] == 'commits':
        continue
    else:
        raise Exception("Unknown operation")
    operations.append({
        "trans_id": trans_id,
        "op": op,
        "x": -1 if (op != 'w' and op != 'r') else int(words[4]),
        "tm": int(words[7])
    })

# Sort operations according to time
operations = sorted(operations, key=lambda x: x['tm'])

print(len(operations))
operations[0]

64066


{'trans_id': 0, 'op': 'r', 'x': 668, 'tm': 1745172180460316}

In [58]:
# Build directed graph

read_set = dict() # [int, set]
write_set = dict() # [int, set]

adj = dict() # [int, set]

for oper in operations:
    tr_id = oper['trans_id']

    if tr_id not in adj:
        adj[tr_id] = set()

    x = oper['x']

    if x != -1:
        if x not in write_set:
            write_set[x] = set()
        if x not in read_set:
            read_set[x] = set()

    op = oper['op']

    if op == 'r':
        # Add conflict edges
        for t in write_set[x]:
            if t != tr_id: adj[t].add((tr_id, x))

        read_set[x].add(tr_id)

    elif op == 'w':
        # Add conflict edges
        for t in write_set[x]:
            if t != tr_id: adj[t].add((tr_id, x))

        for t in read_set[x]:
            if t != tr_id: adj[t].add((tr_id, x))

        write_set[x].add(tr_id)

In [59]:
# Print a few items from adjaceny list
for i, (j, v) in enumerate(adj.items()):
    print(j, v)
    if i > 3:
        break

0 {(864, 6792), (1870, 0), (1993, 534), (1394, 9346), (818, 534), (1798, 6711), (158, 76), (1238, 470), (26, 7556), (637, 345), (846, 345), (734, 3835), (1535, 5327), (1194, 534), (1982, 668), (1372, 3835), (45, 1315), (320, 3835), (1838, 9346), (144, 0), (91, 1315), (1107, 9346), (878, 9346), (1647, 76), (540, 76), (1068, 5327), (1920, 668), (1236, 6711), (850, 1315), (1837, 7556), (1622, 5297), (1996, 9346), (231, 345), (905, 0), (1191, 3835), (790, 5327), (1081, 6788), (1329, 6792), (80, 6711), (609, 9346), (1662, 470), (636, 0), (1173, 345), (1611, 3835), (1598, 0), (1499, 6788), (1369, 6788), (1321, 6711), (374, 0), (1415, 668), (151, 5297), (1985, 76), (185, 6711), (1604, 345), (1153, 0), (897, 6792)}
1 {(502, 3547), (1644, 5793), (80, 5352), (671, 3663), (1986, 3695), (145, 3663), (1049, 7157), (1305, 740), (1017, 3547), (215, 4121), (327, 5793), (1431, 3619), (1453, 3619), (433, 8395), (1090, 1471), (382, 8621), (1542, 8395), (1879, 3547), (699, 8395), (786, 3695), (847, 740), 

In [60]:
# Check for cycle in the graph

def dfs(v, adj, color):
    color[v] = 1
    for (u, _) in adj[v]:
        if color[u] == 0:
            if dfs(u, adj, color):
                return True
        elif color[u] == 1:
            print(v)
            print((u, _))
            return True
    color[v] = 2
    return False

color = dict()
for v in adj.keys():
    color[v] = 0

is_cycle = False

for v in adj.keys():
    if color[v] == 0:
        if dfs(v, adj, color):
            is_cycle = True
            break

if is_cycle: print("Cycle detected")
else: print("No cycle detected")

No cycle detected


In [61]:
1e6 / 31175.981

32.07597541196859